# Titanic dataset: classification

### Description

This notebook sets out an approach for building a survival-classifier predictive model for the Titanic disaster. Various models will be trained and evaluated using data from Kaggle. The best-performer will then be deployed for use. 

Goal: create a machine learning model to generate predictions for whether an individual will survive the titanic disaster.



**Feature descriptors:**
 - Pclass: ticket class
 - Name: full name of passenger
 - Sex: sex (m/f)
 - Age: age in years
 - SibSp: # of siblings/spouses aboard
 - Parch: # of parents / children aboard the Titanic
 - Ticket: ticket number
 - Fare: passenger fare
 - Cabin: cabin number
 - Embarked: port of embarkation
 
 
Target: Survival - whether the individual survived (0 - No, 1 - Yes)

**Import libraries**

In [98]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler, minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, classification_report, auc, roc_auc_score
from sklearn.dummy import DummyClassifier

import pickle
import joblib

**Load datasets**

In [3]:
train_df = pd.read_csv("../datasets/train.csv")
test_df = pd.read_csv("../datasets/test.csv")

**Explore training data**

see Titanic_EDA.ipynb for a more in-depth EDA exercise

In [4]:
# look at the first 10 rows
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# set passenger id as the index
train_df.set_index('PassengerId',inplace=True)
test_df.set_index('PassengerId',inplace=True)

### Feature engineering

The name and ticket columns have been dropped as they contain all unique values and appear unlikely to be useful to the model. The cabin column has a large number of nan values and so this has also been excluded. Then, the remaining rows with nan values have been dropped.

Made the same change on the test dataset.

In [47]:
# name includes title which could be extracted using regex. This could also be an indicator (Mr, Mrs, Miss, Master, Dr., Rev.) 
# for example, indicates whether the passenger is married or not.  
#train_df['Title'] = train_df['Name'].str.extract('([a-zA-Z]{2,}[\.]{1})')
#test_df['Title'] = test_df['Name'].str.extract('([a-zA-Z]{2,}[\.]{1})')

In [6]:
# create new dataframe with cleaned dataset
cleaned_train_df = train_df.drop(columns=['Name','Ticket','Cabin']).dropna(axis=0).copy()
cleaned_test_df = test_df.drop(columns=['Name','Ticket','Cabin']).dropna(axis=0).copy()

In [11]:
# is there another way to use the nan values other than dropping them?
# cabin could be a good indicator as likely encoded the location on the ship of the passenger
# Consider whether to bin fields, like age for example, into categories. Age is unlikley to be useful as a continuous variable. 

**Categoric data**

Use One hot encoding to convert the categoric variables (sex, embarked, parch, Pclass, SibSp) so each category is a feature (1/0) 

This has to be done for both test and training datasets

In [7]:
# one hot encode categoric variables
train_ohe_df = pd.get_dummies(cleaned_train_df,columns=['Sex','Embarked','Parch','Pclass','SibSp'])
test_ohe_df = pd.get_dummies(cleaned_test_df,columns=['Sex','Embarked','Parch','Pclass','SibSp'])

**Numeric data**

Any fields containing continuous numeric data should be scaled or normalized.

In [9]:
# fit and apply a scaler to the training set for the Age variable
age_scaler = StandardScaler().fit(train_ohe_df['Age'].values.reshape(-1, 1))
train_ohe_df['Age'] = age_scaler.transform(train_ohe_df['Age'].values.reshape(-1, 1))

# fit and apply a scaler to the test set for the Fare variable
fare_scaler = StandardScaler().fit(train_ohe_df['Fare'].values.reshape(-1, 1))
train_ohe_df['Fare'] = age_scaler.transform(train_ohe_df['Fare'].values.reshape(-1, 1))

# apply the scaler to the test set
test_ohe_df['Age'] = age_scaler.transform(test_ohe_df['Age'].values.reshape(-1, 1))
test_ohe_df['Fare'] = fare_scaler.transform(test_ohe_df['Fare'].values.reshape(-1, 1))

**Create feature and target datasets**

In [10]:
target_col = ['Survived']
feature_cols=[f for f in train_ohe_df.columns.to_list() if f not in target_col]

X = train_ohe_df[feature_cols].copy()
y = train_ohe_df[target_col].copy()

X_inf = test_ohe_df[feature_cols].copy()

### Model training, validation and testing

Follow the training, validation and testing framework
 - use 60% of the training data for model training
 - use 20% for assessing model performance while fine-tuning parameters
 - hold-out 20% for final evaluation
 
Cross-validation should be applied to the 80% for training and fine-tuning using sklearn's GridSearchCV. This will also allow hyperparameter tuning.

In [12]:
# Split the training data into training and test sets for fitting and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=324)

**Evaluation**: consider what metric should this model be optimised for. 

Create functions for consistent evaluation throughout model experimentation

In [32]:
# create an evaluation function for repetitive evaluation (accuracy only)
def model_eval_acc(model, X_train, y_train, X_test, y_test):
    print(f"Training score: {model.score(X_train,y_train):2.3}")
    print(f"Test score: {model.score(X_test,y_test):2.3}")

Accuracy is being used in this case, with a dummy classifier to benchmark the evaluation with a bad score.

In [33]:
# define a dummy classifier to benchmark the evaluation
dummy_model = DummyClassifier(strategy='most_frequent').fit(X_train,y_train)
model_eval_acc(dummy_model, X_train, y_train, X_test, y_test)

Training score: 0.592
Test score: 0.608


**Train ML model**: train and validate each classifer on the training dataset using cross-validation. Then, score using the evaluation function. 

Create a function for consistent training, validation and testing throughout. 

In [94]:
model_repo = '../models'
def model_train_val(model, params, X_train, y_train, X_test, y_test, cv=4, scoring='accuracy'):
    clf = GridSearchCV(model, param_grid=params, cv=cv, scoring=scoring, return_train_score=True)
    clf.fit(X_train, y_train.values.ravel())
    print(f'Best params: {clf.best_params_}')
    print(f'Best CV score: {clf.best_score_}')
    print(f"Training set score: {clf.cv_results_['mean_train_score'][clf.best_index_]:2.3}")
    print(f'Test set score: {clf.best_estimator_.score(X_test,y_test):2.3}')
    joblib.dump(clf.best_estimator_,os.path.join(model_repo, f'model_{str(clf.best_estimator_)}.pkl'))

Firstly, try kNN, logistic regression and SVM

In [99]:
# k-Nearest Neighbors
params = {'n_neighbors':range(2,11)}
model_train_val(KNeighborsClassifier(), params, X_train, y_train, X_test, y_test)

Best params: {'n_neighbors': 7}
Best CV score: 0.8172338225155126
Training set score: 0.849
Test set score: 0.811


In [96]:
# logistic regression
params = {'penalty':['none','l1','l2'],'C':[0.01,0.1,1,10]}
model_train_val(LogisticRegression(), params, X_train, y_train, X_test, y_test)

Best params: {'C': 1, 'penalty': 'l2'}
Best CV score: 0.7908007485472275
Training set score: 0.816
Test set score: 0.811


In [97]:
# support vector machines
params = {'kernel':('linear','rbf'),'C':(0.01,0.1,1)}
model_train_val(SVC(), params, X_train, y_train, X_test, y_test)

Best params: {'C': 1, 'kernel': 'rbf'}
Best CV score: 0.7767408647690338
Training set score: 0.801
Test set score: 0.804


Try Decision Trees, Random forests and gradient-boosted trees (and XGBoost)

In [100]:
# decision tree
params = {'max_depth':range(1,11),'max_features':range(1,10)}
model_train_val(DecisionTreeClassifier(random_state=0), params, X_train, y_train, X_test, y_test)

Best params: {'max_depth': 7, 'max_features': 8}
Best CV score: 0.8101546340982961
Training set score: 0.9
Test set score: 0.72


In [102]:
# random forests
params = {'n_estimators':range(1,101,10),'max_depth':range(1,11),}
model_train_val(RandomForestClassifier(random_state=0), params, X_train, y_train, X_test, y_test)

Best params: {'max_depth': 10, 'n_estimators': 71}
Best CV score: 0.8154363242391411
Training set score: 0.959
Test set score: 0.762


In [73]:
# gradient boosted trees
params= {'n_estimators':range(1,101,10),'max_depth':range(1,11)}
model_train_val(GradientBoostingClassifier(random_state=0), params, X_train, y_train, X_test, y_test)

Best CV score: 0.8259504579927115
Best params: {'max_depth': 3, 'n_estimators': 91}
Training set score: 0.917
Test set score: 0.79


In [51]:
# using XGBoost
params= {'n_estimators':range(1,101,10),'max_depth':range(1,11),'max_features':range(1,10)}
#model_train_val(XGBClassifier(random_state=0,use_label_encoder=False), params, X_train, y_train, X_test, y_test)

Try using deep learning methods

### Model diagnostic

With the exception of logistic regression, the above scores show that the model is overfitting to the training data in each case. Training set scores are higher than test set scores (high variance so the models are too complex).  

Approaches we can take:
 - Use less features
 - Get more training examples (not possible here)
 - Increase regularization
